In [1]:
!pip install scikit-learn

In [1]:
import pandas as pd

# Store items in Filipino-English DF

# Filipino lines
with open('en-fil.txt/QED.en-fil.fil', 'r') as file:
    fil_lines = file.readlines()

# English lines
with open('en-fil.txt/QED.en-fil.en', 'r') as file:
    en_lines = file.readlines()

combined_items = list(zip(en_lines, fil_lines))
df = pd.DataFrame(combined_items, columns=['English', 'Filipino'])
print(len(df))

42864


In [2]:
from sklearn.model_selection import train_test_split

# Retrieve some matches for fine tuning
df['Split'] = 'unset'
for_translate, for_shots = train_test_split(df, test_size=0.1, random_state=42)
train, test = train_test_split(for_translate, test_size=0.1, random_state=42)

df.loc[for_shots.index, 'Split'] = 'shots'
df.loc[train.index, 'Split'] = 'train'
df.loc[test.index, 'Split'] = 'test'

In [16]:
df.to_csv('eng_to_tgl.csv')

: 

In [4]:
!pip install transformers

In [6]:
!pip install datasets safetensors accelerate

In [3]:
from datasets import Dataset, DatasetDict

# Convert your DataFrame to a Hugging Face Dataset
dataset = Dataset.from_pandas(df)
print(dataset)

dataset_dict = DatasetDict({
    "train": dataset.filter(lambda x: x["Split"] == "train"),
    "test": dataset.filter(lambda x: x["Split"] == "test")
})

# Access train and validation datasets
train_dataset = dataset_dict["train"]
val_dataset = dataset_dict["test"]


/data/students/juan/anaconda3/envs/ai351/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dataset({
    features: ['English', 'Filipino', 'Split'],
    num_rows: 42864
})


Filter: 100%|██████████| 42864/42864 [00:00<00:00, 267461.85 examples/s]


In [8]:
!pip install sentencepiece safetensors
!pip install -U "huggingface_hub[cli]"

In [4]:
import os
from transformers import AutoTokenizer, AutoModelForCausalLM

save_dir = "./models/CuatroLLM"

os.makedirs(save_dir, exist_ok=True)

tokenizer = AutoTokenizer.from_pretrained("britllm/CuatroLLM")
tokenizer.save_pretrained(save_dir)

model = AutoModelForCausalLM.from_pretrained('britllm/CuatroLLM')
model.save_pretrained(save_dir)

tokenizer.pad_token = tokenizer.eos_token

In [ ]:
tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(examples):
    return tokenizer(
        examples["English"],
        text_target=examples["Filipino"],
        padding="max_length",
        truncation=True,
        max_length=1024,
    )

tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_val = val_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/34719 [00:00<?, ? examples/s]

Map: 100%|██████████| 3858/3858 [00:03<00:00, 1170.86 examples/s]


In [9]:
print(tokenized_train[0])

{'English': 'That meal was a religious experience.\n', 'Filipino': 'Sobrang sarap ng niluto mo.\n', 'Split': 'train', 'input_ids': [1, 2193, 592, 284, 471, 263, 12962, 7271, 29889, 13, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,

In [11]:
from transformers import Seq2SeqTrainingArguments

epochs = 1

learning_rate = 5e-5

training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    num_train_epochs=epochs,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=4,
    eval_accumulation_steps=4,
    #gradient_checkpointing=True,
    fp16=True,
    fp16_full_eval=True,

    learning_rate=learning_rate,
    lr_scheduler_type='constant',  # "constant", "linear", "cosine"
    
    eval_strategy="steps",  # or "epoch"
    eval_steps=100,
    save_strategy="epoch",
    logging_steps=50,
    report_to='none',       
)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [12]:
from transformers import Seq2SeqTrainer

# Initialize the trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
)

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [13]:
trainer.train()

/data/students/juan/anaconda3/envs/ai351/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Step,Training Loss,Validation Loss
100,0.019900,0.170203
200,0.018000,0.151811
300,0.019300,0.147515
400,0.017400,0.144643
500,0.017300,0.142301
600,0.016400,0.139314
700,0.017500,0.136925
800,0.016500,0.135299
900,0.018600,0.132774
1000,0.015500,0.132535


TrainOutput(global_step=1085, training_loss=0.03356285496241486, metrics={'train_runtime': 6232.8594, 'train_samples_per_second': 5.57, 'train_steps_per_second': 0.174, 'total_flos': 2.7301729844330496e+17, 'train_loss': 0.03356285496241486, 'epoch': 1.0})

In [14]:
!pip3 install sacrebleu sentencepiece -q

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [15]:
trainer.save_model('models/finetuned_eng_tgl')